### **Importing Required Libraries**

In [225]:
import pandas as pd
import matplotlib as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
import os

### **Loading CSV Data and Creating Data Frame**

In [226]:
data = pd.read_csv(r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\processed\weather data\weather_dataset_train.csv')

In [227]:
data.sample(10)

,is_day,cloud_cover,precipitation,mood
4752,1,10.971036,1.701817,Sad
1974,0,30.565830,0.013352,Happy
3627,0,40.777342,0.303573,Calm
3351,0,7.265632,0.381320,Calm
623,1,8.414192,0.118338,Energetic
971,1,5.045810,0.067355,Energetic
1192,0,3.553802,0.085719,Energetic
336,1,12.050961,0.037027,Energetic
4384,1,32.306189,1.302203,Sad
857,0,11.584436,0.116130,Energetic


### **Vectorizing Data Using Map Function**

In [228]:
dict_mood = {
    'Energetic': 1,
    'Happy': 2,
    'Calm': 3,
    'Sad': 4
}

data['encoded_mood'] = data['mood'].map(dict_mood)
data.sample(10)

,is_day,cloud_cover,precipitation,mood,encoded_mood
4188,0,80.421087,0.561971,Sad,4
2712,0,65.778493,0.044551,Calm,3
796,1,15.423931,0.111576,Energetic,1
697,0,18.251807,0.020116,Energetic,1
4505,0,37.135307,1.460764,Sad,4
756,0,21.859152,0.131584,Energetic,1
3560,1,34.636675,0.414225,Calm,3
1445,1,44.252354,0.024752,Happy,2
3498,1,97.627020,0.109275,Calm,3
167,0,23.161306,0.073121,Energetic,1


In [229]:
# Dividing data frame into features and target on which the model will fit
features = ['is_day', 'cloud_cover', 'precipitation']
target = ['encoded_mood']

X = data[features]
y = data[target]

In [230]:
X.sample(10)

,is_day,cloud_cover,precipitation
1105,1,4.771849,0.049040
3130,0,1.870059,0.482995
2126,1,38.669247,0.026245
1548,0,43.944108,0.069479
971,1,5.045810,0.067355
2868,0,6.140076,0.556033
822,1,23.693406,0.114096
4856,0,64.343082,1.250789
2458,0,33.867422,0.143785
2978,0,25.467989,0.361715


In [231]:
y.sample(10)

,encoded_mood
1019,1
1027,1
1787,2
4712,4
950,1
3200,3
2472,2
4442,4
1654,2
3595,3


### **Vectorizing Test Data Using Map Function and Dividing into X_train and y_test**

In [232]:
# Loading test dataset using pandas
test_data = pd.read_csv(r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\processed\weather data\weather_dataset_test.csv')

# Encoding mood using map() function to vectorize
test_data['encoded_mood'] = test_data['mood'].map(dict_mood)

X_test = test_data.drop('encoded_mood', axis=1)
X_test = X_test.drop('mood', axis=1)

In [233]:
# Viewing test data
X_test.sample(10)

,is_day,cloud_cover,precipitation
1726,1,85.264330,0.872706
1052,1,48.818808,0.560242
1704,0,80.458796,0.639931
1949,1,17.426935,1.387781
1629,0,81.573143,1.684748
1638,1,70.256318,1.085704
937,1,42.880840,0.048382
1863,1,57.762508,0.814922
1058,0,12.052774,0.588157
429,1,7.962980,0.124935


In [234]:
y_test = test_data['encoded_mood']

In [235]:
# Viewing the y_test
y_test

0       1
1       1
2       1
3       1
4       1
       ..
1995    4
1996    4
1997    4
1998    4
1999    4
Name: encoded_mood, Length: 2000, dtype: int64

### **Creating and Training the RandomForestClassifier**

In [236]:
# Creating random forest classifier object
rf = RandomForestClassifier(n_estimators=20, max_depth=10)

In [237]:
# Fitting the training data on random forest classifier
rf.fit(X, y)

c:\Users\njain\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10, n_estimators=20)

### **Predicting the Accuracy**

In [238]:
y_pred = rf.predict(X_test)

In [239]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [240]:
accuracy

0.9975

### **Predicting Mood on Random Value**

In [241]:
# 'is_day, cloud_cover, precipitation'
test_random_value = [0, 50, 0.0]

def get_key_from_value(dict_mood, predicted_mood_EncodedValue):
    for key, value in dict_mood.items():
        if value == predicted_mood_EncodedValue:
            return key
    # If the value is not found, you can handle it accordingly
    return None

predicted_mood_EncodedValue = rf.predict([test_random_value])[0]
predicted_mood = get_key_from_value(dict_mood, predicted_mood_EncodedValue)
print(predicted_mood)

Happy


c:\Users\njain\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### **Saving the Mood Prediction Model from Weather**

In [242]:
# specific path for the model
model_directory = r'C:\Users\njain\OneDrive - Cal State Fullerton\SPRING 2024\CPSC 597 Project\Project\APMBSS\data\models'
model_filename = 'mood_detection_weather_model.pkl'
model_path = os.path.join(model_directory, model_filename)

# Save the model to the specified path
with open(model_path, 'wb') as file:
    pickle.dump(rf, file)